In [1]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark
from pyspark.sql.functions import col

In [2]:
# Import Libraries
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
import pyspark
from pyspark import SQLContext
# Setup the Configuration
conf = pyspark.SparkConf()
# conf.setMaster('spark://csce-nguyen-s4:7077')
spark_context = SparkSession.builder.config(conf=conf).getOrCreate()
sqlcontext = SQLContext(spark_context)
# Setup the Schema
schema = StructType([
StructField("User ID", IntegerType(),True),
StructField("Username", StringType(),True),
StructField("Browser", StringType(),True),
StructField("OS", StringType(),True),
])
# Add Data
data = ([(1580, "Barry", "FireFox", "Windows" ),
(5820, "Sam", "MS Edge", "Linux"),
(2340, "Harry", "Vivaldi", "Windows"),
(7860, "Albert", "Chrome", "Windows"),
(1123, "May", "Safari", "macOS")
])
# Setup the Data Frame
user_data_df = sqlcontext.createDataFrame(data,schema=schema)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-11-28 21:14:09,282 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/grads/t/tiendat.ng.cs/.local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.master", "spark://csce-nguyen-s4:7077") \
    .getOrCreate()

In [4]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', 'csce-nguyen-s4.engr.tamu.edu'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.warehouse.dir',
  'file:/home/grads/t/tiendat.ng.cs/github_repos/PLM_and_BugReport_datasets/spark-warehouse'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1701227649425'),
 ('spark.driver.port', '38109'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.app.startTime', '1701227648517')]

In [5]:
df = spark.read.csv("/itineraries.csv", header=True, inferSchema=True)


In [6]:
df.count()

82138753

In [7]:
df.head(10)

2023-11-28 21:23:11,531 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(legId='9ca0e81111c683bec1012473feefd28f', searchDate='2022-04-16', flightDate='2022-04-17', startingAirport='ATL', destinationAirport='BOS', fareBasisCode='LA0NX0MC', travelDuration='PT2H29M', elapsedDays=0, isBasicEconomy=False, isRefundable=False, isNonStop=True, baseFare=217.67, totalFare=248.6, seatsRemaining=9, totalTravelDistance=947, segmentsDepartureTimeEpochSeconds='1650214620', segmentsDepartureTimeRaw='2022-04-17T12:57:00.000-04:00', segmentsArrivalTimeEpochSeconds='1650223560', segmentsArrivalTimeRaw='2022-04-17T15:26:00.000-04:00', segmentsArrivalAirportCode='BOS', segmentsDepartureAirportCode='ATL', segmentsAirlineName='Delta', segmentsAirlineCode='DL', segmentsEquipmentDescription='Airbus A321', segmentsDurationInSeconds='8940', segmentsDistance='947', segmentsCabinCode='coach'),
 Row(legId='98685953630e772a098941b71906592b', searchDate='2022-04-16', flightDate='2022-04-17', startingAirport='ATL', destinationAirport='BOS', fareBasisCode='LA0NX0MC', travelDuration='P

In [8]:
filtereddf = df.select("legId", "startingAirport").filter((col("elapsedDays") > 0) & (col("seatsRemaining") < 10))

In [9]:
filtereddf.unpersist()

DataFrame[legId: string, startingAirport: string]

In [10]:
filtereddf.count()

12266709

In [11]:
filtereddf.cache()

DataFrame[legId: string, startingAirport: string]

In [12]:
filtereddf.count()

12266709

In [13]:
filtereddf.count()

12266709

In [14]:
filtereddf2 = df.select("legId", "startingAirport").filter((col("elapsedDays") > 1) & (col("seatsRemaining") < 10))

In [15]:
filtereddf2.count()

753

In [21]:
filtereddf3 = df.select("legId", "startingAirport").filter((col("elapsedDays") > 0) & (col("seatsRemaining") < 10))

In [22]:
filtereddf3.count()

12266709

In [25]:
filtereddf4 = filtereddf3.select("startingAirport")

In [26]:
filtereddf4.count()

12266709

In [28]:
filtereddf4.explain()

== Physical Plan ==
InMemoryTableScan [startingAirport#27]
   +- InMemoryRelation [legId#24, startingAirport#27], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) Project [legId#24, startingAirport#27]
            +- *(1) Filter (((isnotnull(elapsedDays#31) AND isnotnull(seatsRemaining#37)) AND (elapsedDays#31 > 0)) AND (seatsRemaining#37 < 10))
               +- FileScan csv [legId#24,startingAirport#27,elapsedDays#31,seatsRemaining#37] Batched: false, DataFilters: [isnotnull(elapsedDays#31), isnotnull(seatsRemaining#37), (elapsedDays#31 > 0), (seatsRemaining#3..., Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://csce-nguyen-s1.engr.tamu.edu:9000/itineraries.csv], PartitionFilters: [], PushedFilters: [IsNotNull(elapsedDays), IsNotNull(seatsRemaining), GreaterThan(elapsedDays,0), LessThan(seatsRem..., ReadSchema: struct<legId:string,startingAirport:string,elapsedDays:int,seatsRemaining:int>




In [29]:
filtereddf3.explain()

== Physical Plan ==
InMemoryTableScan [legId#24, startingAirport#27]
   +- InMemoryRelation [legId#24, startingAirport#27], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) Project [legId#24, startingAirport#27]
            +- *(1) Filter (((isnotnull(elapsedDays#31) AND isnotnull(seatsRemaining#37)) AND (elapsedDays#31 > 0)) AND (seatsRemaining#37 < 10))
               +- FileScan csv [legId#24,startingAirport#27,elapsedDays#31,seatsRemaining#37] Batched: false, DataFilters: [isnotnull(elapsedDays#31), isnotnull(seatsRemaining#37), (elapsedDays#31 > 0), (seatsRemaining#3..., Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://csce-nguyen-s1.engr.tamu.edu:9000/itineraries.csv], PartitionFilters: [], PushedFilters: [IsNotNull(elapsedDays), IsNotNull(seatsRemaining), GreaterThan(elapsedDays,0), LessThan(seatsRem..., ReadSchema: struct<legId:string,startingAirport:string,elapsedDays:int,seatsRemaining:int>




In [32]:
filtereddf5 = filtereddf3.filter((col("startingAirport") == "ATL"))

In [33]:
filtereddf5.explain()

== Physical Plan ==
*(1) Filter (isnotnull(startingAirport#27) AND (startingAirport#27 = ATL))
+- InMemoryTableScan [legId#24, startingAirport#27], [isnotnull(startingAirport#27), (startingAirport#27 = ATL)]
      +- InMemoryRelation [legId#24, startingAirport#27], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(1) Project [legId#24, startingAirport#27]
               +- *(1) Filter (((isnotnull(elapsedDays#31) AND isnotnull(seatsRemaining#37)) AND (elapsedDays#31 > 0)) AND (seatsRemaining#37 < 10))
                  +- FileScan csv [legId#24,startingAirport#27,elapsedDays#31,seatsRemaining#37] Batched: false, DataFilters: [isnotnull(elapsedDays#31), isnotnull(seatsRemaining#37), (elapsedDays#31 > 0), (seatsRemaining#3..., Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://csce-nguyen-s1.engr.tamu.edu:9000/itineraries.csv], PartitionFilters: [], PushedFilters: [IsNotNull(elapsedDays), IsNotNull(seatsRemaining), GreaterThan(elapsedDays,0), LessThan(seatsRem..

In [34]:
filtereddf5.count()

491326

In [36]:
filtereddf2.explain()

== Physical Plan ==
*(1) Project [legId#24, startingAirport#27]
+- *(1) Filter (((isnotnull(elapsedDays#31) AND isnotnull(seatsRemaining#37)) AND (elapsedDays#31 > 1)) AND (seatsRemaining#37 < 10))
   +- FileScan csv [legId#24,startingAirport#27,elapsedDays#31,seatsRemaining#37] Batched: false, DataFilters: [isnotnull(elapsedDays#31), isnotnull(seatsRemaining#37), (elapsedDays#31 > 1), (seatsRemaining#3..., Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://csce-nguyen-s1.engr.tamu.edu:9000/itineraries.csv], PartitionFilters: [], PushedFilters: [IsNotNull(elapsedDays), IsNotNull(seatsRemaining), GreaterThan(elapsedDays,1), LessThan(seatsRem..., ReadSchema: struct<legId:string,startingAirport:string,elapsedDays:int,seatsRemaining:int>


